In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
raw_data = pd.read_csv('data.csv')

In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30360 entries, 0 to 30359
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 9144 non-null   object
 1   Price                 30360 non-null  object
 2   Address               30360 non-null  object
 3   Price per m2          12 non-null     object
 4   Area                  30358 non-null  object
 5   Bedrooms              20632 non-null  object
 6   Toilets               19720 non-null  object
 7   Published At          30360 non-null  object
 8   Diện tích             30358 non-null  object
 9   Mức giá               30360 non-null  object
 10  Hướng nhà             7640 non-null   object
 11  Hướng ban công        4677 non-null   object
 12  Ngày đăng             30360 non-null  object
 13  Ngày hết hạn          30360 non-null  object
 14  Loại tin              30360 non-null  object
 15  Mã tin                30360 non-null

In [4]:
unduplicated_data = raw_data.drop_duplicates(subset=['Mã tin'], keep='first')
unduplicated_data.loc[unduplicated_data[['Developer', 'Status']].isnull().all(axis=1), ['Developer', 'Status']] = ['cá nhân/môi giới', 'Đã bàn giao']
unduplicated_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12446 entries, 0 to 30359
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 3838 non-null   object
 1   Price                 12446 non-null  object
 2   Address               12446 non-null  object
 3   Price per m2          1 non-null      object
 4   Area                  12445 non-null  object
 5   Bedrooms              8202 non-null   object
 6   Toilets               7805 non-null   object
 7   Published At          12446 non-null  object
 8   Diện tích             12445 non-null  object
 9   Mức giá               12446 non-null  object
 10  Hướng nhà             3131 non-null   object
 11  Hướng ban công        1828 non-null   object
 12  Ngày đăng             12446 non-null  object
 13  Ngày hết hạn          12446 non-null  object
 14  Loại tin              12446 non-null  object
 15  Mã tin                12446 non-null

C:\Users\hienb\AppData\Local\Temp\ipykernel_32744\1051689887.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unduplicated_data.loc[unduplicated_data[['Developer', 'Status']].isnull().all(axis=1), ['Developer', 'Status']] = ['cá nhân/môi giới', 'Đã bàn giao']


In [5]:
unduplicated_data = unduplicated_data.drop(["Price per m2","Project Address", "Title", "Diện tích", "Mức giá", "Project Title", "Ngày hết hạn", "Number of Buildings", "Ngày đăng", "Mã tin"], axis="columns")
unduplicated_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12446 entries, 0 to 30359
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Price                 12446 non-null  object
 1   Address               12446 non-null  object
 2   Area                  12445 non-null  object
 3   Bedrooms              8202 non-null   object
 4   Toilets               7805 non-null   object
 5   Published At          12446 non-null  object
 6   Hướng nhà             3131 non-null   object
 7   Hướng ban công        1828 non-null   object
 8   Loại tin              12446 non-null  object
 9   Status                12446 non-null  object
 10  Number of Apartments  3838 non-null   object
 11  Developer             12446 non-null  object
dtypes: object(12)
memory usage: 1.2+ MB


In [ ]:
unduplicated_data = unduplicated_data[unduplicated_data["Price"] != "Thỏa thuận"]

def clean_area(area):
    if isinstance(area, str):
        area = area.replace(' m²', '').replace('.', '').replace(',', '.')
    return pd.to_numeric(area, errors='coerce')

unduplicated_data['Area'] = unduplicated_data['Area'].astype(str).apply(clean_area)

num_bins = int(np.ceil(np.log2(len(unduplicated_data)) + 1))
unduplicated_data['Area Range'] = pd.cut(unduplicated_data['Area'], bins=num_bins)

toilets_df = unduplicated_data[['Area', 'Toilets', 'Area Range']].copy()
bedrooms_df = unduplicated_data[['Area', 'Bedrooms', 'Area Range']].copy()

# Extract numeric values from 'Toilets' and 'Bedrooms' for mean calculation
toilets_df['Toilets'] = toilets_df['Toilets'].str.extract('(\d+)')[0].astype(float)
bedrooms_df['Bedrooms'] = bedrooms_df['Bedrooms'].str.extract('(\d+)')[0].astype(float)

toilets_avg = toilets_df.groupby('Area Range')['Toilets'].mean()
bedrooms_avg = bedrooms_df.groupby('Area Range')['Bedrooms'].mean()

def assign_values(row, col, avg_values):
    if pd.isna(row[col]) and not pd.isna(row['Area Range']):
        return avg_values.get(row['Area Range'], np.nan)  # Ensure it handles cases where the area range is out of bounds
    else:
        return row[col]

unduplicated_data['Toilets'] = unduplicated_data.apply(lambda row: assign_values(row, 'Toilets', toilets_avg), axis=1)
unduplicated_data['Bedrooms'] = unduplicated_data.apply(lambda row: assign_values(row, 'Bedrooms', bedrooms_avg), axis=1)

def convert_to_string_with_suffix(value):
    if pd.notna(value):
        # If the value is in the format 'X phòng', we want to extract the numeric part
        numeric_value = ''.join(filter(str.isdigit, str(value)))  # Extract digits only
        if numeric_value:
            return int(numeric_value)
    return np.nan

unduplicated_data['Toilets'] = unduplicated_data['Toilets'].apply(convert_to_string_with_suffix)
unduplicated_data['Bedrooms'] = unduplicated_data['Bedrooms'].apply(convert_to_string_with_suffix)

# Print info about the final dataframe
print("\nFinal DataFrame info:")
unduplicated_data.info()



Final DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11508 entries, 0 to 30359
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Price                 11508 non-null  object  
 1   Address               11508 non-null  object  
 2   Area                  11506 non-null  float64 
 3   Bedrooms              11496 non-null  float64 
 4   Toilets               11496 non-null  float64 
 5   Published At          11508 non-null  object  
 6   Hướng nhà             2957 non-null   object  
 7   Hướng ban công        1725 non-null   object  
 8   Loại tin              11508 non-null  object  
 9   Status                11508 non-null  object  
 10  Number of Apartments  3606 non-null   object  
 11  Developer             11508 non-null  object  
 12  Area Range            11506 non-null  category
dtypes: category(1), float64(3), object(9)
memory usage: 1.2+ MB


In [7]:
unduplicated_data.head(10)

,Price,Address,Area,Bedrooms,Toilets,Published At,Hướng nhà,Hướng ban công,Loại tin,Status,Number of Apartments,Developer,Area Range
0,"15,2 tỷ","Đường Dương Bá Trạc, Phường 2, Quận 8, Hồ Chí ...",40.0,6.0,4.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]"
1,"5,6 tỷ","Screc Tower, Đường Trường Sa, Phường 12, Quận ...",90.0,3.0,2.000000e+00,23/10/2024,NaN,NaN,Tin thường,đã bàn giao,60-106 m²,Công ty CP Đầu tư Xây dựng và Kinh doanh nhà S...,"(-272.993, 18673.2]"
2,7 tỷ,"Dự án HaDo Centrosa Garden, Đường 3/2, Phường ...",87.0,2.0,2.000000e+00,23/10/2024,NaN,NaN,Tin VIP Kim Cương,Đã bàn giao,2.178,Tập đoàn Hà Đô,"(-272.993, 18673.2]"
3,"12,35 tỷ","Đường Nguyễn Trãi, Phường 7, Quận 5, Hồ Chí Minh",60.0,4.0,3.843741e+15,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]"
5,"1,35 tỷ","Dự án Dream Home Riverside, Đường Nguyễn Văn L...",57.0,2.0,2.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đang mở bán,55-75 m²,Công ty TNHH SX TM Lý Khương,"(-272.993, 18673.2]"
6,"7,8 tỷ","Đường Phạm Văn Bạch, Phường 15, Tân Bình, Hồ C...",100.0,3.0,3.000000e+00,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]"
7,"5,95 tỷ","Dự án ICON 56, Đường Bến Vân Đồn, Phường 12, Q...",87.0,3.0,2.000000e+00,15/10/2024,NaN,NaN,Tin thường,Đã bàn giao,47-112 m²,Novaland Group,"(-272.993, 18673.2]"
9,16 tỷ,"Phường Phước Long B, Quận 9, Hồ Chí Minh",156.0,8.0,9.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]"
10,"8,6 tỷ","Đường Lê Văn Sỹ, Phường 12, Quận 3, Hồ Chí Minh",34.0,3.0,4.000000e+00,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]"
11,"8,7 tỷ","Đường Huỳnh Tấn Phát, Phường Phú Thuận, Quận 7...",80.0,4.0,5.000000e+00,04/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]"


In [8]:
# Chuyển đổi dữ liệu ở cột Area sang float
def convert_area_value(value):
    try: 
        # Chỉ thực hiện thay thế nếu value là chuỗi
        if isinstance(value, str):
            # Lọc bỏ các dấu chấm ở số hàng nghìn
            value = value.replace('m²', '').replace('.', '')

            # Thay dấu phẩy thành dấu chấm
            value = value.replace(',', '.')
            
            return float(value)
        
        # Nếu value đã là số (float hoặc int), trả về trực tiếp
        elif isinstance(value, (int, float)):
            return float(value)

    except Exception as e:
        print(f"Lỗi khi xử lý giá trị: {value}. Chi tiết lỗi: {e}")
        return None

unduplicated_data['Diện Tích'] = unduplicated_data['Area'].apply(convert_area_value)

In [9]:
unduplicated_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11508 entries, 0 to 30359
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Price                 11508 non-null  object  
 1   Address               11508 non-null  object  
 2   Area                  11506 non-null  float64 
 3   Bedrooms              11496 non-null  float64 
 4   Toilets               11496 non-null  float64 
 5   Published At          11508 non-null  object  
 6   Hướng nhà             2957 non-null   object  
 7   Hướng ban công        1725 non-null   object  
 8   Loại tin              11508 non-null  object  
 9   Status                11508 non-null  object  
 10  Number of Apartments  3606 non-null   object  
 11  Developer             11508 non-null  object  
 12  Area Range            11506 non-null  category
 13  Diện Tích             11506 non-null  float64 
dtypes: category(1), float64(4), object(9)
memory usage: 1.

In [10]:
# Hàm chuyển đổi giá trị thành tỷ với xử lý trường hợp "triệu/m²"
def convert_to_billion(value, area):
    try:
        # Chỉ thực hiện thay thế nếu value là chuỗi
        if isinstance(value, str):
            # Thay dấu phẩy thành dấu chấm
            value = value.replace(',', '.')

            # Dữ liệu chỉ đánh giá các bds rao bán nên sẽ không xử lý các bds cho thuê
            if '/tháng' in value:
                return None
            
            # Kiểm tra nếu có "triệu/m²"
            elif 'triệu/m²' in value:
                price_per_sqm = float(value.replace(' triệu/m²', '')) / 1000  # Đổi từ triệu/m² thành tỷ/m²
                return price_per_sqm * area  # Nhân với diện tích để có giá trị tổng

            # Kiểm tra nếu có "triệu/m²"
            elif 'tỷ/m²' in value:
                price_per_sqm = float(value.replace(' tỷ/m²', ''))
                return price_per_sqm * area  # Nhân với diện tích để có giá trị tổng

            # Xử lý các trường hợp có đơn vị "tỷ" hoặc "triệu"
            elif 'tỷ' in value:
                return float(value.replace(' tỷ', ''))
            
            # Chuyển "triệu" thành "tỷ"
            elif 'triệu' in value:
                return float(value.replace(' triệu', '')) / 1000  
        
        # Nếu value đã là số (float hoặc int), trả về trực tiếp
        elif isinstance(value, (int, float)):
            return float(value)
        
        return None
        
    except Exception as e:
        print(f"Lỗi khi xử lý giá trị: {value}. Chi tiết lỗi: {e}")
        return None

# Áp dụng hàm vào cột 'giá_cả' và truyền thêm cột 'diện_tích' để tính giá trị cuối cùng
unduplicated_data['Giá'] = unduplicated_data.apply(lambda row: convert_to_billion(row['Price'], row['Diện Tích']), axis=1)

In [11]:
unduplicated_data = unduplicated_data.dropna(subset=["Giá"])
unduplicated_data.head()

,Price,Address,Area,Bedrooms,Toilets,Published At,Hướng nhà,Hướng ban công,Loại tin,Status,Number of Apartments,Developer,Area Range,Diện Tích,Giá
0,"15,2 tỷ","Đường Dương Bá Trạc, Phường 2, Quận 8, Hồ Chí ...",40.0,6.0,4.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",40.0,15.20
1,"5,6 tỷ","Screc Tower, Đường Trường Sa, Phường 12, Quận ...",90.0,3.0,2.000000e+00,23/10/2024,NaN,NaN,Tin thường,đã bàn giao,60-106 m²,Công ty CP Đầu tư Xây dựng và Kinh doanh nhà S...,"(-272.993, 18673.2]",90.0,5.60
2,7 tỷ,"Dự án HaDo Centrosa Garden, Đường 3/2, Phường ...",87.0,2.0,2.000000e+00,23/10/2024,NaN,NaN,Tin VIP Kim Cương,Đã bàn giao,2.178,Tập đoàn Hà Đô,"(-272.993, 18673.2]",87.0,7.00
3,"12,35 tỷ","Đường Nguyễn Trãi, Phường 7, Quận 5, Hồ Chí Minh",60.0,4.0,3.843741e+15,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",60.0,12.35
5,"1,35 tỷ","Dự án Dream Home Riverside, Đường Nguyễn Văn L...",57.0,2.0,2.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đang mở bán,55-75 m²,Công ty TNHH SX TM Lý Khương,"(-272.993, 18673.2]",57.0,1.35


In [12]:
unduplicated_data["Triệu/m2"] = (unduplicated_data["Giá"] / unduplicated_data["Diện Tích"] * 100).round(1)
unduplicated_data.head(10)

,Price,Address,Area,Bedrooms,Toilets,Published At,Hướng nhà,Hướng ban công,Loại tin,Status,Number of Apartments,Developer,Area Range,Diện Tích,Giá,Triệu/m2
0,"15,2 tỷ","Đường Dương Bá Trạc, Phường 2, Quận 8, Hồ Chí ...",40.0,6.0,4.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",40.0,15.20,38.0
1,"5,6 tỷ","Screc Tower, Đường Trường Sa, Phường 12, Quận ...",90.0,3.0,2.000000e+00,23/10/2024,NaN,NaN,Tin thường,đã bàn giao,60-106 m²,Công ty CP Đầu tư Xây dựng và Kinh doanh nhà S...,"(-272.993, 18673.2]",90.0,5.60,6.2
2,7 tỷ,"Dự án HaDo Centrosa Garden, Đường 3/2, Phường ...",87.0,2.0,2.000000e+00,23/10/2024,NaN,NaN,Tin VIP Kim Cương,Đã bàn giao,2.178,Tập đoàn Hà Đô,"(-272.993, 18673.2]",87.0,7.00,8.0
3,"12,35 tỷ","Đường Nguyễn Trãi, Phường 7, Quận 5, Hồ Chí Minh",60.0,4.0,3.843741e+15,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",60.0,12.35,20.6
5,"1,35 tỷ","Dự án Dream Home Riverside, Đường Nguyễn Văn L...",57.0,2.0,2.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đang mở bán,55-75 m²,Công ty TNHH SX TM Lý Khương,"(-272.993, 18673.2]",57.0,1.35,2.4
6,"7,8 tỷ","Đường Phạm Văn Bạch, Phường 15, Tân Bình, Hồ C...",100.0,3.0,3.000000e+00,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",100.0,7.80,7.8
7,"5,95 tỷ","Dự án ICON 56, Đường Bến Vân Đồn, Phường 12, Q...",87.0,3.0,2.000000e+00,15/10/2024,NaN,NaN,Tin thường,Đã bàn giao,47-112 m²,Novaland Group,"(-272.993, 18673.2]",87.0,5.95,6.8
9,16 tỷ,"Phường Phước Long B, Quận 9, Hồ Chí Minh",156.0,8.0,9.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",156.0,16.00,10.3
10,"8,6 tỷ","Đường Lê Văn Sỹ, Phường 12, Quận 3, Hồ Chí Minh",34.0,3.0,4.000000e+00,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",34.0,8.60,25.3
11,"8,7 tỷ","Đường Huỳnh Tấn Phát, Phường Phú Thuận, Quận 7...",80.0,4.0,5.000000e+00,04/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",80.0,8.70,10.9


In [13]:
unduplicated_data["Ngày đăng"] = pd.to_datetime(unduplicated_data["Published At"], format="%d/%m/%Y")
unduplicated_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11498 entries, 0 to 30359
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Price                 11498 non-null  object        
 1   Address               11498 non-null  object        
 2   Area                  11498 non-null  float64       
 3   Bedrooms              11488 non-null  float64       
 4   Toilets               11488 non-null  float64       
 5   Published At          11498 non-null  object        
 6   Hướng nhà             2956 non-null   object        
 7   Hướng ban công        1723 non-null   object        
 8   Loại tin              11498 non-null  object        
 9   Status                11498 non-null  object        
 10  Number of Apartments  3601 non-null   object        
 11  Developer             11498 non-null  object        
 12  Area Range            11498 non-null  category      
 13  Diện Tích       

In [14]:
def convert_the_number_of_rooms(value):
    try:
        if isinstance(value, str):
            if "phòng" in value:
                value = value.replace(" phòng", "")
                return float(value)
            else: return None
        elif isinstance (value, (int, float)):
            return float(value)
        return None
    except Exception as e:
        print(f"Lỗi khi xử lý giá trị {value}, chi tiết lỗi {e}")
        return None
    
unduplicated_data["Số Phòng Ngủ"] = unduplicated_data["Bedrooms"].apply(convert_the_number_of_rooms)
unduplicated_data["Số Nhà Vệ Sinh"] = unduplicated_data["Toilets"].apply(convert_the_number_of_rooms)

In [15]:
# unduplicated_data.head(10)
unduplicated_data.head(10)

,Price,Address,Area,Bedrooms,Toilets,Published At,Hướng nhà,Hướng ban công,Loại tin,Status,Number of Apartments,Developer,Area Range,Diện Tích,Giá,Triệu/m2,Ngày đăng,Số Phòng Ngủ,Số Nhà Vệ Sinh
0,"15,2 tỷ","Đường Dương Bá Trạc, Phường 2, Quận 8, Hồ Chí ...",40.0,6.0,4.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",40.0,15.20,38.0,2024-10-17,6.0,4.000000e+00
1,"5,6 tỷ","Screc Tower, Đường Trường Sa, Phường 12, Quận ...",90.0,3.0,2.000000e+00,23/10/2024,NaN,NaN,Tin thường,đã bàn giao,60-106 m²,Công ty CP Đầu tư Xây dựng và Kinh doanh nhà S...,"(-272.993, 18673.2]",90.0,5.60,6.2,2024-10-23,3.0,2.000000e+00
2,7 tỷ,"Dự án HaDo Centrosa Garden, Đường 3/2, Phường ...",87.0,2.0,2.000000e+00,23/10/2024,NaN,NaN,Tin VIP Kim Cương,Đã bàn giao,2.178,Tập đoàn Hà Đô,"(-272.993, 18673.2]",87.0,7.00,8.0,2024-10-23,2.0,2.000000e+00
3,"12,35 tỷ","Đường Nguyễn Trãi, Phường 7, Quận 5, Hồ Chí Minh",60.0,4.0,3.843741e+15,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",60.0,12.35,20.6,2024-10-22,4.0,3.843741e+15
5,"1,35 tỷ","Dự án Dream Home Riverside, Đường Nguyễn Văn L...",57.0,2.0,2.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đang mở bán,55-75 m²,Công ty TNHH SX TM Lý Khương,"(-272.993, 18673.2]",57.0,1.35,2.4,2024-10-17,2.0,2.000000e+00
6,"7,8 tỷ","Đường Phạm Văn Bạch, Phường 15, Tân Bình, Hồ C...",100.0,3.0,3.000000e+00,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",100.0,7.80,7.8,2024-10-22,3.0,3.000000e+00
7,"5,95 tỷ","Dự án ICON 56, Đường Bến Vân Đồn, Phường 12, Q...",87.0,3.0,2.000000e+00,15/10/2024,NaN,NaN,Tin thường,Đã bàn giao,47-112 m²,Novaland Group,"(-272.993, 18673.2]",87.0,5.95,6.8,2024-10-15,3.0,2.000000e+00
9,16 tỷ,"Phường Phước Long B, Quận 9, Hồ Chí Minh",156.0,8.0,9.000000e+00,17/10/2024,NaN,NaN,Tin VIP Bạc,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",156.0,16.00,10.3,2024-10-17,8.0,9.000000e+00
10,"8,6 tỷ","Đường Lê Văn Sỹ, Phường 12, Quận 3, Hồ Chí Minh",34.0,3.0,4.000000e+00,22/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",34.0,8.60,25.3,2024-10-22,3.0,4.000000e+00
11,"8,7 tỷ","Đường Huỳnh Tấn Phát, Phường Phú Thuận, Quận 7...",80.0,4.0,5.000000e+00,04/10/2024,NaN,NaN,Tin thường,Đã bàn giao,NaN,cá nhân/môi giới,"(-272.993, 18673.2]",80.0,8.70,10.9,2024-10-04,4.0,5.000000e+00


In [16]:
gbd_path = './gbd.csv'
df_gbd = pd.read_csv(gbd_path)
df_gbd['Tên đường'] = df_gbd['TÊN ĐƯỜNG'].str.lower()
df_gbd['Quận'] = df_gbd['QUẬN'].str.lower()
# Giá trung bình của đất nơi dự án
def calculate_avg_district_prices(df_gbd):
    district_avg_prices = {}

    # Group by district and calculate the average of 'Giá đất đề nghị điều chỉnh'
    for district in df_gbd['Quận'].unique():
        # Filter data for the district
        district_data = df_gbd[df_gbd['Quận'] == district].copy()
        
        # Ensure 'Giá đất đề nghị điều chỉnh' is numeric and calculate the average
        district_data.loc[:, 'Giá đất đề nghị điều chỉnh'] = pd.to_numeric(district_data['Giá đất đề nghị điều chỉnh'], errors='coerce')
        
        # Calculate the average price for the district
        avg_price = district_data['Giá đất đề nghị điều chỉnh'].mean()
        district_avg_prices[district] = avg_price

    return district_avg_prices

district_avg_prices = calculate_avg_district_prices(df_gbd)
district_avg_prices

{'bình chánh': 25803.571428571428,
 'bình thạnh': 105035.10638297872,
 'bình tân': 45781.513083048914,
 'cần giờ': 7238.851851851853,
 'củ chi': 8990.760869565218,
 'gò vấp': 70582.75862068965,
 'hóc môn': 20194.915254237287,
 'nhà bè': 32532.196969696968,
 'phú nhuận': 124477.19298245614,
 'quận 1': 267213.53383458644,
 'quận 10': 131008.0,
 'quận 11': 115061.30136986301,
 'quận 12': 44303.08641975309,
 'quận 2': 59327.65113974232,
 'quận 3': 213742.85714285713,
 'quận 4': 138849.38271604938,
 'quận 5': 139679.38144329897,
 'quận 6': 92683.33333333333,
 'quận 7': 80628.373015873,
 'quận 8': 61907.186544342505,
 'quận 9': 59327.65113974232,
 'thủ đức': 59327.65113974232,
 'tân bình': 101933.79629629629,
 'tân phú': 74354.52755905512}

In [17]:
def contains_hcm_and_district(address, district):
    # Ensure that we are checking if the district and either Hồ Chí Minh or Thủ Đức appear in the address
    return district.lower() in address.lower() and ('hồ chí minh' in address.lower() or 'thủ đức' in address.lower())

def assign_district_price(row):
    # Normalize address to lowercase
    address = row['Address'].lower()
    street_match = df_gbd[df_gbd['Tên đường'].apply(lambda x: x.lower() in address)]
    print(f"Processing address: {address}, {len(street_match)}")  # Debug print for address
    
    for idx, match in street_match.iterrows():
        street_name = match['Tên đường'].lower()
        district = match['Quận'].lower()

        if district in ['Thủ Đức']:
            print(f"District {district} is a sub-district of Thủ Đức, checking street match...")
            price = match['Giá đất đề nghị điều chỉnh']
            return 'thủ đức', price, street_name

        if street_name in address and district in address:
            print(f"Found exact match: Street = {street_name}, District = {district}")
            price = match['Giá đất đề nghị điều chỉnh']
            return district, price, street_name

    for district in district_avg_prices:
        if contains_hcm_and_district(address, district):
            price = district_avg_prices[district]
            print(f"Price found for district {district}: {price}")
            return district, price, None

    print(f"No match found for address: {address}")
    return None, None, None

unduplicated_data[['Quận', 'Giá Đề Xuất', 'Đường']] = unduplicated_data.apply(lambda row: pd.Series(assign_district_price(row)), axis=1)
unduplicated_data = unduplicated_data[unduplicated_data['Giá Đề Xuất'].notna()]
unduplicated_data.head(10000).to_csv('new_data.csv', index=False)

Processing address: đường dương bá trạc, phường 2, quận 8, hồ chí minh, 2
Found exact match: Street = dương bá trạc, District = quận 8
Processing address: screc tower, đường trường sa, phường 12, quận 3, hồ chí minh, 3
Found exact match: Street = trường sa, District = quận 3
Processing address: dự án hado centrosa garden, đường 3/2, phường 12, quận 10, hồ chí minh, 1
Price found for district quận 1: 267213.53383458644
Processing address: đường nguyễn trãi, phường 7, quận 5, hồ chí minh, 3
Found exact match: Street = nguyễn trãi, District = quận 5
Processing address: dự án dream home riverside, đường nguyễn văn linh, phường 7, quận 8, hồ chí minh, 4
Found exact match: Street = nguyễn văn linh, District = quận 8
Processing address: đường phạm văn bạch, phường 15, tân bình, hồ chí minh, 3
Found exact match: Street = phạm văn bạch, District = tân bình
Processing address: dự án icon 56, đường bến vân đồn, phường 12, quận 4, hồ chí minh, 5
Found exact match: Street = bến vân đồn, District = 

TypeError: can only insert Interval objects and NA into an IntervalArray

In [ ]:
# Xử lý số lượng các căn hộ
def handle_number_of_apartment(value):
    try:
        if isinstance(value, str):
            value = value.replace(".", "")
            value = int(value)
            return value
        # Nếu value đã là số (float hoặc int), trả về trực tiếp
        elif isinstance(value, (int, float)):
            return float(value)
        
        return None        
    except Exception as e:
        print(f"Lỗi khi xử lý giá trị: {value}. Chi tiết lỗi: {e}")
        return None
    
    

unduplicated_data["Số Căn Hộ"] = unduplicated_data["Number of Apartments"].apply(handle_number_of_apartment)

In [ ]:
unduplicated_data.info()

In [ ]:
unduplicated_data.describe()

Có các oulier tại cột price như 900 tỷ, cột area như 30,000 tỷ, bedroom 200 phòng, 96 toilet --> không phải dự án bất động sản riêng lẻ --> chuỗi dự án căn hộ cấp cao? --> Loại bỏ

In [ ]:
# Code xử lý loại bỏ các ngoại lai quá lớn có thể ảnh hưởng đến dữ liệu
# Tính Q1, Q3 và IQR của cột Giá
Q1_price = unduplicated_data['Giá'].quantile(0.25)
Q3_price = unduplicated_data['Giá'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Tính Q1, Q3 và IQR của cột Diện Tích
Q1_area = unduplicated_data['Diện Tích'].quantile(0.25)
Q3_area = unduplicated_data['Diện Tích'].quantile(0.75)
IQR_area = Q3_area - Q1_area


# Lọc các giá trị không phải là ngoại lai
df_no_outliers = unduplicated_data[(unduplicated_data['Diện Tích'] >= Q1_area - 1.5 * IQR_area) & (unduplicated_data['Diện Tích'] <= Q3_area + 1.5 * IQR_area)]
df_no_outliers = df_no_outliers[(df_no_outliers['Giá'] >= Q1_price - 1.5 * IQR_price) & (df_no_outliers['Giá'] <= Q3_price + 1.5 * IQR_price)]

In [ ]:
# Việt hóa
unduplicated_data["Chủ Đầu Tư"] = unduplicated_data["Developer"]
unduplicated_data["Trạng Thái"] = unduplicated_data["Status"]

In [ ]:
# Drop các cột dư thừa sau khi Việt hóa
unduplicated_data = unduplicated_data.drop(["Area", "Price", "Address", "Bedrooms", "Toilets", "Published At", "Developer", "Status", "Number of Apartments"], axis="columns")

In [ ]:
unduplicated_data.info()

## THỐNG KÊ SỐ LƯỢNG BDS MỖI QUẬN, HUYỆN

In [ ]:
# Đếm số lượng BĐS trong mỗi quận
district_counts = unduplicated_data['Quận'].value_counts().reset_index()
district_counts.columns = ['Quận', 'Số lượng BĐS']

# Vẽ biểu đồ với seaborn
plt.figure(figsize=(10, 6))
sns.barplot(data=district_counts, x='Quận', y='Số lượng BĐS', palette='viridis')
plt.title('Số lượng BĐS ở mỗi quận')
plt.xlabel('Quận')
plt.ylabel('Số lượng BĐS')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## SỰ PHÂN BỔ VỀ GIÁ TRỊ BDS

In [ ]:


# Vẽ biểu đồ với seaborn
plt.figure(figsize=(10, 6))
sns.histplot(data=df_no_outliers, x="Giá", bins=40, kde=True)
plt.title('Sự phân bổ giá trị BDS tại thành phố Hồ Chí Minh')
plt.xlabel('Giá trị (tỷ VND)')
plt.ylabel('Số lượng BĐS')
plt.tight_layout()
plt.show()

## SỰ LIÊN HỆ GIỮA GIÁ TRỊ VÀ DIỆN TÍCH BDS TẠI TPHCM

In [ ]:

# Vẽ biểu đồ với seaborn
plt.figure(figsize=(10, 6))
sns.regplot(data=df_no_outliers, x=df_no_outliers["Diện Tích"], y=df_no_outliers["Giá"])
plt.title('Sự liên hệ giữa Giá trị và Diện tích BDS tại thành phố Hồ Chí Minh')
plt.xlabel('Diện tích (m2)')
plt.ylabel('Giá (tỷ VND)')
plt.tight_layout()
plt.show()

Nhận xét: 
- Giá và Diện tích của BDS có tính tương quan thuận với nhau.
- Biểu đồ cho thấy sự phân tán lớn chứng tỏ giá bất động sản ở mỗi khu vực khác nhau có sự khác biệt lớn.

## Sự liên hệ giữa Giá và Diện tích BDS tại một số quận ở thành phố Hồ Chí Minh

In [ ]:
# Rút trích dữ liệu của một số quận để Visualization 
district_2 = df_no_outliers[df_no_outliers["Quận"] == "quận 2"]
district_7 = df_no_outliers[df_no_outliers["Quận"] == "quận 7"]
district_9 = df_no_outliers[df_no_outliers["Quận"] == "quận 9"]
district_BinhThanh = df_no_outliers[df_no_outliers["Quận"] == "Bình Thạnh"]

In [ ]:
datasets = [district_2, district_7, district_9, district_BinhThanh]
district_names = ["Quận 2", "Quận 7", "Quận 9", "Bình Thạnh"]
n_rows = 2
n_cols = 2

fig, ax = plt.subplots(n_rows, n_cols, figsize = (n_cols * 4, n_rows * 4))

for r in range(n_rows):
    for c in range(n_cols):
        i = r *  n_cols + c
        if i < len(datasets):
            ax_i = ax[r, c]
            sns.regplot(data=datasets[i], x=datasets[i]["Diện Tích"], y=datasets[i]["Giá"], ax=ax_i)
            ax_i.set_title(f"Biểu đồ {i + 1}: {district_names[i]}")
plt.tight_layout()
plt.show()


In [ ]:
num_devs = unduplicated_data["Chủ Đầu Tư"].nunique()
num_devs

#### Có tổng số 189 nhà đầu tư BDS khác nhau trong tập dữ liệu